In [1]:
#load libraries and setup file
import sys,os
root_path = os.path.abspath(os.path.join('..'))
print(root_path)
if root_path not in sys.path:
    sys.path.append(root_path)
%run -i "../lib/utils/ipynb_setup.py"
%load_ext autoreload
%autoreload 2
%matplotlib inline

#plotting
import matplotlib.pyplot as pl
import seaborn as sns
sns.set_style("ticks", {'axes.grid' : True})
# pl.rc("figure", facecolor="white",figsize = (8,8))
pl.rc("figure", facecolor="gray",figsize = (8,8))
pl.rc('text', usetex=True)
pl.rc('text.latex', preamble=r'\usepackage{amsmath}')
pl.rc('lines',markeredgewidth = 2)
pl.rc('font',size = 12)

/home/mptouzel/Dropbox/scripts/Projects/topicmod/code


## load carbon tax data

In [3]:
df=pd.read_stata('../../data/carbon_tax/CCOP_wave1.dta')
df.head(1)

,responseid,status,intcode,interview_start,interview_end,temptime,lang,prov,d1,d3,...,d7,d8,d10,d11,d12,d13,d14,immig,kids,Weight
0,27,complete,C1,2019-02-22 11:27:04,2019-02-22 11:54:52,1665,EN,ON,Male,63,...,0,2,"Within a suburb, adjacent to a large city",8,College / CEGEP graduate,This doesn't apply to me,"$40,000-$59,999","No, Born in Canada",No,2.389879


Select data to analyze

In [40]:
carbon_tax_question='po3a'
participant_sex='d1'
# french_response_ids=df[df.lang=='FR'].index

question_id_name=carbon_tax_question
meta_data_names=[participant_sex,'lang']
exclusions=[] # french_response_ids #english only

resp_data=df.drop(columns=exclusions).loc[:,[question_id_name]+meta_data_names].copy()

Translate French to English

In [41]:
from lib.lib import translate_responses
resp_data=translate_responses(resp_data)

translating 293 non-empty responses of 558 french responses
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
26

Run custom corpus cleaning

In [19]:
import nltk
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mptouzel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
from lib.lib import clean_corpus
occurs_in_morethan=1
wordfixfile='../../data/carbon_tax/CCOP_wave1_nonwords.csv'
customclean_data=clean_corpus(resp_data,carbon_tax_question,wordfixfile=wordfixfile,occurs_in_morethan=occurs_in_morethan)
customclean_data[0].head()

,po3a,d1,lang,po3aclean,po3aclean_stems,po3aclean_stems_wordvec,po3aclean_stems_wordvec_keep,po3aclean_stems_keep,po3aclean_stems_keepwordvec
6,It will get people to think about their consum...,Male,EN,"[get, people, think, consumption, perhaps, cut...","[get, peopl, think, consumpt, perhap, cut, bac...","[494, 852, 1197, 234, 857, 266, 99, 900, 706, ...","[494, 852, 1197, 234, 857, 266, 99, 900, 706, ...","[get, peopl, think, consumpt, perhap, cut, bac...","[266, 464, 652, 127, 466, 148, 53, 496, 396, 1..."
7,Phasing out the use of fossil fuels is essenti...,Male,EN,"[phasing, use, fossil, fuels, essential, save,...","[phase, use, fossil, fuel, essenti, save, huma...","[863, 1259, 472, 478, 391, 1036, 564, 945, 418...","[863, 1259, 472, 478, 391, 1036, 564, 706, 472...","[phase, use, fossil, fuel, essenti, save, huma...","[468, 678, 253, 256, 209, 574, 316, 396, 253, ..."
9,People will respect it more and be less wasteful,Female,EN,"[people, respect, less, wasteful]","[peopl, respect, less, wast]","[852, 1001, 668, 1281]","[852, 668, 1281]","[peopl, less, wast]","[464, 371, 690]"
11,I believe putting price or tax on fossil fuels...,Male,EN,"[believe, putting, price, tax, fossil, fuels, ...","[believ, put, price, tax, fossil, fuel, help, ...","[117, 939, 900, 1177, 472, 478, 546, 261, 417,...","[117, 939, 900, 1177, 472, 478, 546, 261, 417,...","[believ, put, price, tax, fossil, fuel, help, ...","[63, 522, 496, 640, 253, 256, 304, 144, 226, 3..."
14,"As price increases, people are more open to op...",Male,EN,"[price, increases, people, open, options, opti...","[price, increas, peopl, open, option, option, ...","[900, 598, 852, 806, 813, 813, 112, 28, 210]","[900, 598, 852, 806, 813, 813, 112, 28]","[price, increas, peopl, open, option, option, ...","[496, 336, 464, 445, 447, 447, 60, 15]"


# STM

## Preprocessing

In R, so need to interface

In [46]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
# for row in rpy2.situation.iter_info():
#     print(row)
#N.b. need to install all packages to get stm 1.3.6 working in R first
# # R library imports
# for __pkg__ in ['Matrix', 'stringr', 'splines', 'matrixStats', 
#     'slam', 'lda', 'glmnet', 'magrittr','stm']:
#     try:
#         robjects.r("library(" + __pkg__ + ")")
#     except:
#         raise Exception("Missing R library " + __pkg__ + 
#             ". Please ensure the R requirements are satisfied.")

In [47]:
robjects.r("library('stm')");

load data into R

In [48]:
from rpy2.robjects.conversion import localconverter

In [50]:
with localconverter(robjects.default_converter + pandas2ri.converter):
    robjects.globalenv['doc_df'] = robjects.conversion.py2rpy(resp_data)
robjects.globalenv['doc_df']

,po3a,d1,lang
0,,Male,EN
1,,Male,EN
2,,Female,EN
3,,Male,EN
4,,Male,EN
...,...,...,...
3308,,Male,EN
3309,,Male,EN
3310,Great,Male,EN
3311,hopefully it does help to reduce climate change,Female,EN


Run STM's corpus cleaning

In [51]:
robjects.r("processed_corpus_temp = textProcessor(doc_df$" + question_id_name +", metadata=doc_df, lowercase=TRUE)");
robjects.r(
    'processed_corpus = prepDocuments(processed_corpus_temp$documents,'+\
                                 'processed_corpus_temp$vocab,'+\
                                 'processed_corpus_temp$meta,'+\
                                 'lower.thresh='+str(occurs_in_morethan)+') \n'+\
    'rm(processed_corpus_temp); invisible(gc())'
);

Building corpus... 
Converting to Lower Case... 
Removing punctuation... 
Removing stopwords... 
Removing numbers... 
Stemming... 
Creating Output... 
Removing 789 of 1508 terms (789 of 10357 tokens) due to frequency 
Removing 17 Documents with No Words 
Your corpus now has 1295 documents, 719 terms and 9568 tokens.

In [52]:
preprocessing_output=dict(robjects.r.processed_corpus.items())
print(list(preprocessing_output.keys()))
stmclean_dictionary=pd.Series(list(preprocessing_output['vocab']),name='word')
stmclean_corpus=pd.DataFrame.from_records([doc for doc in preprocessing_output['documents'].items()],index=[0]).drop(0,axis=1)
stmclean_corpus.index.name= None
stmclean_corpus.rename(columns={1: question_id_name+'clean_stems_keepwordvec'},inplace=True)
stmclean_corpus[question_id_name+'clean_stems_wordvec']=stmclean_corpus[question_id_name+'clean_stems_keepwordvec'].apply(lambda x:x[0])
extra_columns=dict(preprocessing_output['meta'].items())
for column_name,column in extra_columns.items():
    stmclean_corpus[column_name]=pd.Series(list(column),index=stmclean_corpus.index)
stmclean_data=(stmclean_corpus,stmclean_dictionary)

['documents', 'vocab', 'meta', 'words.removed', 'docs.removed', 'tokens.removed', 'wordcounts']


## compare the two corpii

In [54]:
for tmpdata in [customclean_data,stmclean_data]:
    corpus,worddict=tmpdata
    print('n_resp={}, n_dict={}'.format(len(corpus),len(worddict)))
    print(list(corpus.columns.values))
    corpus.po3aclean_stems_keepwordvec.head()

n_resp=1340, n_dict=711
['po3a', 'd1', 'lang', 'po3aclean', 'po3aclean_stems', 'po3aclean_stems_wordvec', 'po3aclean_stems_wordvec_keep', 'po3aclean_stems_keep', 'po3aclean_stems_keepwordvec']
n_resp=1295, n_dict=719
['po3aclean_stems_keepwordvec', 'po3aclean_stems_wordvec', 'po3a', 'd1', 'lang']


## Define model & inference parameters

In [55]:
K=5
init_type="Spectral"
seed=None
max_em_its=500
emtol=1e-5
verbose=False#True
reportevery=5
LDAbeta=True
interactions=True
ngroups=1
gamma_prior="Pooled"
sigma_prior=0
kappa_prior="L1"
# model_parameters={
#     'K': 10,  # number of topics
#     'LDAbeta': LDAbeta,
#     'gamma_prior':,
#     'sigma_prior':,
#     'kappa_prior':
# }
# inference_options = {
#     'init_type': init_type,
#     'seed':,
#     'max_em_its':,
#     'emtol':,
#     'verbose':,
#     'ngroups':1,
# }

## Run inference (no prevalence or content)

Initialize

In [21]:
# for __f__ in ['checkBeta.R', 'checkFactors.R', 'cloud.R', 'dmr.R', 
#     'estimateEffect.R', 'exclusivity.R', 'findThoughts.R', 'findTopic.R', 
#     'heldout.R', 'jeffreyskappa.R', 'labelTopics.R', 'manyTopics.R', 
#     'multiSTM.R', 'parseFormulas.R', 'permute.R', 'plot.estimateEffect.R', 
#     'plot.searchK.R', 'plot.STM.R', 'plotModels.R', 'plotQuote.R', 
#     'plotRemoved.R', 'plottingutilfns.R', 'plotTopicLoess.R', 'prepDocuments.R',
#     'produce_cmatrix.R', 'RcppExports.R', 'readCorpus.R', 'residuals.R', 's.R',
#     'sageLabels.R', 'searchK.R', 'selectModel.R', 'semanticCoherence.R', 
#     'simBetas.R', 'spectral.R', 'stm.control.R', 'stm.R', 'STMconvergence.R', 
#     'STMestep.R', 'STMfunctions.R', 'STMinit.R', 'STMlncpp.R', 'STMmnreg.R', 
#     'STMmu.R', 'STMoptbeta.R', 'STMreport.R', 'STMsigma.R', 'summary.STM.R', 
#     'tau.R', 'textProcessor.R', 'thetaPosterior.R', 'toLDAvis.R', 'topicCorr.R', 
#     'topicLasso.R', 'topicQuality.R', 'writeLdac.R']:
#     robjects.r("source('/home/mptouzel/R/x86_64-pc-linux-gnu-library/4.1/stm/R/"+__f__+"')")

In [56]:
yield_seed = lambda x: robjects.NULL if x==None else x
robjects.globalenv["fit"] = robjects.r.stm(
    robjects.r("processed_corpus$documents"), 
    robjects.r("processed_corpus$vocab"), 
    K=K, data=robjects.r("processed_corpus$meta"),
    init_type=init_type, seed=yield_seed(seed),max_em_its=max_em_its)
#, emtol=emtol,
#     verbose=verbose, reportevery=reportevery,
#     LDAbeta=LDAbeta, interactions=interactions,
#     ngroups=ngroups, gamma_prior=gamma_prior,
#     sigma_prior=sigma_prior, kappa_prior=kappa_prior)

Beginning Spectral Initialization 
	 Calculating the gram matrix...
	 Finding anchor words...
 	.....
	 Recovering initialization...
 	.......
Initialization complete.
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 1 (approx. per word bound = -5.930) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 2 (approx. per word bound = -5.744, relative change = 3.135e-02) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 3 (approx. per word bound = -5.667, relative change = 1.339e-02) 
...........................................................................................................
Completed E-Step (

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 29 (approx. per word bound = -5.566, relative change = 5.365e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 30 (approx. per word bound = -5.565, relative change = 5.080e-05) 
Topic 1: help, less, way, support, must 
 Topic 2: energi, pay, altern, pollut, better 
 Topic 3: will, fuel, fossil, make, price 
 Topic 4: use, tax, carbon, chang, emiss 
 Topic 5: peopl, reduc, climat, need, think 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 31 (approx. per word bound = -5.565, relative change = 4.829e-05) 
............................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 57 (approx. per word bound = -5.560, relative change = 3.775e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 58 (approx. per word bound = -5.560, relative change = 3.871e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 59 (approx. per word bound = -5.560, relative change = 3.991e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 60 (approx. per word bound = -5.559, relative change = 4.131e-05) 
Topic 1: help, l

Topic 1: help, less, way, support, good 
 Topic 2: energi, pay, altern, pollut, better 
 Topic 3: will, fuel, fossil, make, price 
 Topic 4: use, tax, carbon, chang, emiss 
 Topic 5: peopl, reduc, climat, need, think 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 86 (approx. per word bound = -5.554, relative change = 3.036e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 87 (approx. per word bound = -5.554, relative change = 3.110e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 88 (approx. per word bound = -5.554, relative change = 3.201e-05) 
............................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 114 (approx. per word bound = -5.549, relative change = 1.765e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 115 (approx. per word bound = -5.549, relative change = 1.691e-05) 
Topic 1: help, less, way, support, good 
 Topic 2: energi, pay, altern, pollut, better 
 Topic 3: will, fuel, fossil, make, price 
 Topic 4: use, tax, carbon, chang, emiss 
 Topic 5: peopl, reduc, climat, need, think 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 116 (approx. per word bound = -5.549, relative change = 1.630e-05) 
.........................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 142 (approx. per word bound = -5.547, relative change = 1.179e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 143 (approx. per word bound = -5.547, relative change = 1.169e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 144 (approx. per word bound = -5.547, relative change = 1.163e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 145 (approx. per word bound = -5.547, relative change = 1.161e-05) 
Topic 1: hel

Topic 1: help, less, way, support, good 
 Topic 2: energi, pay, altern, pollut, better 
 Topic 3: will, fuel, fossil, make, price 
 Topic 4: use, tax, carbon, chang, emiss 
 Topic 5: peopl, reduc, climat, need, think 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 171 (approx. per word bound = -5.545, relative change = 2.503e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 172 (approx. per word bound = -5.544, relative change = 2.626e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 173 (approx. per word bound = -5.544, relative change = 2.733e-05) 
.........................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 199 (approx. per word bound = -5.541, relative change = 1.449e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 200 (approx. per word bound = -5.541, relative change = 1.478e-05) 
Topic 1: help, less, way, support, good 
 Topic 2: energi, pay, altern, pollut, better 
 Topic 3: will, fuel, use, fossil, make 
 Topic 4: tax, carbon, chang, emiss, environ 
 Topic 5: peopl, reduc, need, climat, think 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 201 (approx. per word bound = -5.540, relative change = 1.537e-05) 
.......................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 227 (approx. per word bound = -5.536, relative change = 2.801e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 228 (approx. per word bound = -5.536, relative change = 2.644e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 229 (approx. per word bound = -5.536, relative change = 2.507e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 230 (approx. per word bound = -5.536, relative change = 2.396e-05) 
Topic 1: hel

Topic 1: help, less, way, support, good 
 Topic 2: energi, pay, altern, pollut, better 
 Topic 3: will, use, fuel, fossil, make 
 Topic 4: tax, carbon, chang, climat, emiss 
 Topic 5: peopl, reduc, need, think, encourag 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 256 (approx. per word bound = -5.532, relative change = 2.506e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 257 (approx. per word bound = -5.532, relative change = 2.308e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 258 (approx. per word bound = -5.532, relative change = 2.177e-05) 
......................................

In [57]:
print(list(robjects.r.fit.names))

['mu', 'sigma', 'beta', 'settings', 'vocab', 'convergence', 'theta', 'eta', 'invsigma', 'time', 'version']


In [95]:
nwords=7
test1=robjects.r('labelTopics(fit,n='+str(nwords)+')')
topicwords=pd.Series(np.asarray(test1.rx[1][0]).reshape((nwords,K)).T.tolist(),index=range(1,K+1))
topicwords.index.name='topicid'
print(topicwords.values)
robjects.r('plot(fit)')

double assumed number of topics

In [113]:
yield_seed = lambda x: robjects.NULL if x==None else x
robjects.globalenv["fit"] = robjects.r.stm(
    robjects.r("processed_corpus$documents"), 
    robjects.r("processed_corpus$vocab"), 
    K=2*K, data=robjects.r("processed_corpus$meta"),
    init_type=init_type, seed=yield_seed(seed),max_em_its=max_em_its)

Beginning Spectral Initialization 
	 Calculating the gram matrix...
	 Finding anchor words...
 	..........
	 Recovering initialization...
 	.......
Initialization complete.
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 1 (approx. per word bound = -6.090) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 2 (approx. per word bound = -5.881, relative change = 3.440e-02) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 3 (approx. per word bound = -5.768, relative change = 1.912e-02) 
...........................................................................................................
Completed E-S

 Topic 4: use, fossil, pollut, support, must 
 Topic 5: fuel, will, emiss, dont, save 
 Topic 6: peopl, chang, environ, better, find 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, develop 
 Topic 9: climat, need, cost, get, effect 
 Topic 10: carbon, will, less, futur, caus 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 26 (approx. per word bound = -5.555, relative change = 1.327e-04) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 27 (approx. per word bound = -5.554, relative change = 1.181e-04) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 28 (approx. per word 

Topic 1: tax, compani, consumpt, increas, forc 
 Topic 2: help, way, sourc, protect, warm 
 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fossil, pollut, support, must 
 Topic 5: fuel, will, emiss, dont, save 
 Topic 6: peopl, chang, environ, better, find 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, develop 
 Topic 9: climat, need, cost, get, effect 
 Topic 10: carbon, will, less, caus, futur 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 51 (approx. per word bound = -5.545, relative change = 4.502e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 52 (approx. per word bound = -5.544, relative change = 4.125e-05) 
..............................................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 75 (approx. per word bound = -5.539, relative change = 2.978e-05) 
Topic 1: tax, compani, consumpt, increas, forc 
 Topic 2: help, way, protect, sourc, warm 
 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fossil, pollut, support, must 
 Topic 5: fuel, will, emiss, dont, save 
 Topic 6: peopl, chang, environ, better, find 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, develop 
 Topic 9: climat, need, cost, get, effect 
 Topic 10: carbon, less, will, caus, futur 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 76 (approx. per word bound = -5.539, relative change = 2.831e-05) 
..............................................................

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 99 (approx. per word bound = -5.536, relative change = 2.166e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 100 (approx. per word bound = -5.536, relative change = 2.000e-05) 
Topic 1: tax, compani, consumpt, increas, forc 
 Topic 2: help, way, protect, sourc, warm 
 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fossil, pollut, support, must 
 Topic 5: will, fuel, emiss, dont, save 
 Topic 6: peopl, chang, environ, better, find 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, develop 
 Topic 9: climat, need, cost, get, effect 
 Topic 10: carbon, less, will, caus, futur 
.............................................................

...

 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, gas 
 Topic 9: chang, climat, need, cost, get 
 Topic 10: carbon, less, caus, futur, believ 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 196 (approx. per word bound = -5.515, relative change = 4.526e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 197 (approx. per word bound = -5.515, relative change = 4.796e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 198 (approx. per word bound = -5.515, relative change = 4.975e-05) 
........................................................................................

 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fossil, fuel, pollut, support 
 Topic 5: will, emiss, fuel, dont, save 
 Topic 6: peopl, environ, better, find, good 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, amount 
 Topic 9: chang, climat, need, cost, get 
 Topic 10: carbon, less, caus, futur, believ 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 221 (approx. per word bound = -5.509, relative change = 3.702e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 222 (approx. per word bound = -5.509, relative change = 3.599e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-S

Completed M-Step. 
Completing Iteration 245 (approx. per word bound = -5.506, relative change = 1.463e-05) 
Topic 1: tax, compani, consumpt, increas, industri 
 Topic 2: help, way, protect, sourc, warm 
 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fuel, fossil, pollut, support 
 Topic 5: will, emiss, dont, save, much 
 Topic 6: peopl, environ, better, find, good 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, amount 
 Topic 9: chang, climat, need, cost, get 
 Topic 10: carbon, less, caus, futur, believ 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 246 (approx. per word bound = -5.506, relative change = 1.468e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 247 (approx. per 

Completing Iteration 269 (approx. per word bound = -5.503, relative change = 2.376e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 270 (approx. per word bound = -5.503, relative change = 2.511e-05) 
Topic 1: tax, compani, consumpt, increas, industri 
 Topic 2: help, way, protect, sourc, warm 
 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fuel, fossil, pollut, support 
 Topic 5: will, emiss, dont, save, much 
 Topic 6: peopl, environ, better, find, good 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, amount 
 Topic 9: chang, climat, need, cost, get 
 Topic 10: carbon, less, caus, futur, product 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 271 (approx. per word bound = -5.50

...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 294 (approx. per word bound = -5.500, relative change = 1.897e-05) 
...........................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 295 (approx. per word bound = -5.499, relative change = 1.953e-05) 
Topic 1: tax, compani, consumpt, increas, industri 
 Topic 2: help, way, protect, sourc, warm 
 Topic 3: energi, make, pay, encourag, planet 
 Topic 4: use, fuel, fossil, pollut, support 
 Topic 5: will, emiss, dont, save, much 
 Topic 6: peopl, environ, better, find, good 
 Topic 7: altern, money, can, put, like 
 Topic 8: reduc, think, price, hope, drive 
 Topic 9: chang, climat, need, cost, get 
 Topic 10: carbon, less, caus, futur, product 
.........................................................

In [116]:
nwords=7
test1=robjects.r('labelTopics(fit,n='+str(nwords)+')')
topicwords=pd.Series(np.asarray(test1.rx[1][0]).reshape((nwords,2*K)).T.tolist(),index=range(1,2*K+1))
topicwords.index.name='topicid'
print(topicwords.values)
robjects.r('plot(fit)')

[list(['tax', 'compani', 'consumpt', 'increas', 'industri', 'forc', 'incent'])
 list(['help', 'way', 'protect', 'sourc', 'warm', 'govern', 'global'])
 list(['energi', 'make', 'pay', 'encourag', 'planet', 'work', 'may'])
 list(['use', 'fuel', 'fossil', 'pollut', 'support', 'must', 'transport'])
 list(['will', 'emiss', 'dont', 'save', 'much', 'decreas', 'tri'])
 list(['peopl', 'environ', 'better', 'find', 'good', 'someth', 'car'])
 list(['altern', 'money', 'can', 'put', 'like', 'develop', 'option'])
 list(['reduc', 'think', 'price', 'hope', 'drive', 'amount', 'gas'])
 list(['chang', 'climat', 'need', 'cost', 'get', 'effect', 'believ'])
 list(['carbon', 'less', 'caus', 'futur', 'product', 'just', 'harm'])]


<rpy2.rinterface_lib.sexp.NULLType object at 0x7f4a655dbd80> [RTYPES.NILSXP]

In [117]:
robjects.r('plot(fit,type="perspectives", topics=c(1,8))')

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f4a655dbd80> [RTYPES.NILSXP]

## code junkyard

In [43]:
# Finish initialization
robjects.r('''

    # Extract objects
    documents <- fit$documents
    vocab <- fit$vocab
    settings <- fit$settings 
    model <- fit$model
    verbose <- settings$verbose

    # Initialize parameters
    ngroups <- settings$ngroups
    if(is.null(model)) {

        if(verbose) cat("Beginning Initialization.\n")
        model <- stm.init(documents, settings)

        # If using the Lee and Mimno method of setting K, update the settings
        if(settings$dim$K==0) settings$dim$K <- nrow(model$beta[[1]])

        # Unpack
        mu <- list(mu=model$mu)
        sigma <- model$sigma
        beta <- list(beta=model$beta)
        if(!is.null(model$kappa)) beta$kappa <- model$kappa
        lambda <- model$lambda
        convergence <- NULL 

        # Discard the old object
        # rm(model)
    } else {

        if(verbose) cat("Restarting Model...\n")

        # Extract from a standard STM object so we can simply continue
        mu <- model$mu
        beta <- list(beta=lapply(model$beta$logbeta, exp))
        if(!is.null(model$beta$kappa)) beta$kappa <- model$beta$kappa
        sigma <- model$sigma
        lambda <- model$eta
        convergence <- model$convergence

        # Manually declare the model not converged or it will stop after the first iteration
        convergence$stopits <- FALSE
        convergence$converged <- FALSE

        # Iterate by 1 as that would have happened otherwise
        convergence$its <- convergence$its + 1 
    }    

    #Pull out some book keeping elements
    ntokens <- sum(settings$dim$wcounts$x)
    betaindex <- settings$covariates$betaindex
    stopits <- FALSE
    if(ngroups!=1) {
        groups <- cut(1:length(documents), breaks=ngroups, labels=FALSE) 
    }
    suffstats <- vector(mode="list", length=ngroups)
''');

# Get the settings
fit = dict(zip(robjects.globalenv['fit'].names, 
         list(robjects.globalenv['fit'])))
settings = dict(zip( fit['settings'].names, 
         list(fit['settings'])))
modK, modA, modV, modN = [int(settings['dim'][i][0]) for i in range(4)]

# Some setup for EM, retrieving the R objects
stopits = False
documents = [np.array(x) for x in list(robjects.globalenv['documents'])]
beta_index = np.array(robjects.globalenv['betaindex'])
beta = [np.array(x) for x in robjects.globalenv['beta'][0]]
Lambda = np.array(robjects.globalenv['lambda'])
def rmu_2py(mu):
    out = {'mu': np.array(mu[0])}
    if len(mu) > 1:
        out['gamma'] = np.array(mu[1])
    else:
        out['gamma'] = None
    return out
mu = rmu_2py(robjects.globalenv['mu'])
sigma = np.array(robjects.globalenv['sigma'])
verbose = settings['verbose'][0]

# Clear the convergence object
try:
    del convergence
    i = 0
except:
    i = 0


R[write to console]: Error in rep(1:length(documents), times = VsubD) : 
  invalid 'times' argument



RRuntimeError: Error in rep(1:length(documents), times = VsubD) : 
  invalid 'times' argument


Run training loop

In [ ]:
# Initiate timer
start_time = time.clock()

# Run EM!
while stopits is not True:

    # Non-blocked updates
    if ngroups==1:

        # Run the E-step
        sigma_ss, beta_ss, bound_ss, Lambda = self.__estep_spark__(documents, beta_index, beta, 
                                                          Lambda, mu, sigma, verbose, False)
        if verbose:
            print("Completed E-Step, Iteration " +  str(i+1))

        # Run the M-step
        mu = self.__rmu_2py__(robjects.globalenv['opt.mu'](Lambda, 
                             robjects.r("settings$gamma$mode"),
                             robjects.r("settings$covariates$X"),
                             robjects.r("settings$gamma$enet")))
        sigma = np.array(robjects.r("opt.sigma")(sigma_ss, 
                                                 Lambda, 
                                                 mu['mu'], 
                                                 float(robjects.r("settings$sigma$prior"))))
        beta_new_temp = robjects.globalenv['opt.beta'](self.__pybeta_ss_2r__(beta_ss), 
                                                       robjects.r('beta$kappa'), 
                                                       robjects.r('settings'))
        beta = [np.array(x) for x in beta_new_temp[0]]

        if verbose:
            print("Completed M-Step, Iteration " +  str(i+1))

    # Check for convergence
    try:
        convergence = robjects.globalenv['convergence.check'](bound_ss, convergence, 
                                                              robjects.globalenv['settings'])
    except:
        convergence = robjects.globalenv['convergence.check'](bound_ss, 
                                                              robjects.globalenv['convergence'], 
                                                              robjects.globalenv['settings'])

    # Print updates if we have not converged
    stopits = bool(self.__rlist_2py__(convergence)['stopits'][0])
    if stopits is not True and verbose:
        robjects.r.report(convergence, robjects.r.ntokens, self.__pybeta_2r__(beta),
                 robjects.r.vocab, robjects.r('settings$topicreportevery'),
                 verbose);

    # Increase counter
    i += 1
    if verbose:
        print("")

# Some book-keeping
self.em_time = time.clock() - start_time
self.trained = True

# Save the results
self.beta = beta
self.logbeta = [np.log(x) for x in self.beta]
self.Lambda = Lambda
self.mu = mu
self.sigma = sigma
self.settings = settings
self.vocab = robjects.r.vocab
self.convergence = convergence
self.theta = np.array(robjects.r("theta_from_lambda")(Lambda))
self.eta = np.array(robjects.r("eta_from_lambda")(Lambda))
self.invsigma = np.linalg.inv(sigma)

In [ ]:
def train(self,data, document_col, K, prevalence=None, content=None, init_type="Spectral", 
    seed=None, max_em_its=500, emtol=1e-5, verbose=True, reportevery=5, 
    LDAbeta=True, interactions=True, ngroups=1, gamma_prior="Pooled", 
    sigma_prior=0, kappa_prior="L1"):
    '''
    Train a STM model

    Parameters:
      data - Pandas dataframe
        The data, in the form of a Pandas dataframe

      document_col - string
        Name of the data column containing the raw text 

      K - int 
        Number of topics

      prevalence - string | list of strings
        Name or list of names of columns containing prevalence covariates

      content - string | list of strings
        Name or list of names of columns containing content covariates 

      init_type - string
        The method of initialization. Must be either Latent Dirichlet Allocation 
        ("LDA"), "Random" or "Spectral".

      seed - int
        Random seed

      max_em_its - int 
        The maximum number of EM iterations. If convergence has not been met 
        at this point, a message will be printed.

      emtol - float
        Convergence tolerance. EM stops when the relative change in the approximate 
        bound drops below this level. Defaults to .00001.

      verbose - bool
        A logical flag indicating whether information should be printed to the 
        screen. During the E-step (iteration over documents) a dot will print 
        each time one percent of the documents are completed. At the end of each 
        iteration the approximate bound will also be printed.

      reportevery - int
        An integer determining the intervals at which labels are printed to 
        the screen during fitting. Defaults to every 5 iterations.

      LDAbeta - bool
        A logical that defaults to True when there are no content covariates. 
        When set to FALSE the model performs SAGE style topic updates (sparse 
        deviations from a baseline).

      interactions - bool
        A logical that defaults to TRUE. This automatically includes interactions 
        between content covariates and the latent topics. Setting it to FALSE 
        reduces to a model with no interactive effects.

      ngroups - int
        Number of groups for memoized inference.

      gamma_prior - string
        Sets the prior estimation method for the prevalence covariate model. 
        The default Pooled options uses Normal prior distributions with a 
        topic-level pooled variance which is given a broad gamma hyperprior. 
        The alternative L1 uses glmnet to estimate a grouped penalty between L1-L2.

      sigma_prior - int
        A scalar between 0 and 1 which defaults to 0. This sets the strength of 
        regularization towards a diagonalized covariance matrix. Setting the value 
        above 0 can be useful if topics are becoming too highly correlated.

      kappa_prior - int
        Sets the prior estimation for the content covariate coefficients. 
        The default option is the L1 prior. The second option is Jeffreys 
        which is markedly less computationally efficient but is included for 
        backwards compatability.
    '''

    # Blocked updates are not yet supported
    if ngroups > 1:
        raise Exception("Blocked inference not yet supported. ")

    # Push the data to R
    pandas2ri.activate()
    try:
        robjects.globalenv['doc_df'] = pandas2ri.py2ri(data)
    except:
        raise Exception("The data must be in the form of a Pandas dataframe")

    # Parse any date objects
    robjects.r('''
        if("date" %in% colnames(doc_df)){
            doc_df$date = as.Date(doc_df$date)
        }
    ''');

    # Prep the corpus
    print("Beginning document preprocessing.")
    robjects.r("processed_corpus_temp = textProcessor(doc_df$" + document_col + 
        ", metadata=doc_df, lowercase=TRUE)");
    robjects.r('''
        processed_corpus = prepDocuments(processed_corpus_temp$documents,
                                     processed_corpus_temp$vocab, 
                                     processed_corpus_temp$meta,
                                     lower.thresh=1)
        rm(processed_corpus_temp); invisible(gc())
    ''');
    print("")

    # Seed parsing
    yield_seed = lambda x: robjects.NULL if x==None else x

    # Initialize the run
    if prevalence != None and content != None:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, prevalence=robjects.Formula(prevalence),
            content=robjects.Formula(content),
            data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)
    elif prevalence != None and content == None:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, prevalence=robjects.Formula(prevalence),
            data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)
    elif prevalence == None and content != None:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, content=robjects.Formula(content),
            data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)
    else:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)

    # Finish initialization
    robjects.r('''

        # Extract objects
        documents <- fit$documents
        vocab <- fit$vocab
        settings <- fit$settings 
        model <- fit$model
        verbose <- settings$verbose

        # Initialize parameters
        ngroups <- settings$ngroups
        if(is.null(model)) {

            if(verbose) cat("Beginning Initialization.\n")
            model <- stm.init(documents, settings)

            # If using the Lee and Mimno method of setting K, update the settings
            if(settings$dim$K==0) settings$dim$K <- nrow(model$beta[[1]])

            # Unpack
            mu <- list(mu=model$mu)
            sigma <- model$sigma
            beta <- list(beta=model$beta)
            if(!is.null(model$kappa)) beta$kappa <- model$kappa
            lambda <- model$lambda
            convergence <- NULL 

            # Discard the old object
            # rm(model)
        } else {

            if(verbose) cat("Restarting Model...\n")

            # Extract from a standard STM object so we can simply continue
            mu <- model$mu
            beta <- list(beta=lapply(model$beta$logbeta, exp))
            if(!is.null(model$beta$kappa)) beta$kappa <- model$beta$kappa
            sigma <- model$sigma
            lambda <- model$eta
            convergence <- model$convergence

            # Manually declare the model not converged or it will stop after the first iteration
            convergence$stopits <- FALSE
            convergence$converged <- FALSE

            # Iterate by 1 as that would have happened otherwise
            convergence$its <- convergence$its + 1 
        }    

        #Pull out some book keeping elements
        ntokens <- sum(settings$dim$wcounts$x)
        betaindex <- settings$covariates$betaindex
        stopits <- FALSE
        if(ngroups!=1) {
            groups <- cut(1:length(documents), breaks=ngroups, labels=FALSE) 
        }
        suffstats <- vector(mode="list", length=ngroups)
    ''');

    # Get the settings
    fit = dict(zip(robjects.globalenv['fit'].names, 
             list(robjects.globalenv['fit'])))
    settings = dict(zip( fit['settings'].names, 
             list(fit['settings'])))
    self.K, self.A, self.V, self.N = [int(settings['dim'][i][0]) for i in range(4)]

    # Some setup for EM, retrieving the R objects
    stopits = False
    documents = [np.array(x) for x in list(robjects.globalenv['documents'])]
    beta_index = np.array(robjects.globalenv['betaindex'])
    beta = [np.array(x) for x in robjects.globalenv['beta'][0]]
    Lambda = np.array(robjects.globalenv['lambda'])
    mu = self.__rmu_2py__(robjects.globalenv['mu'])
    sigma = np.array(robjects.globalenv['sigma'])
    verbose = settings['verbose'][0]

    # Clear the convergence object
    try:
        del convergence
        i = 0
    except:
        i = 0

    # Initiate timer
    start_time = time.clock()

    # Run EM!
    while stopits is not True:

        # Non-blocked updates
        if ngroups==1:

            # Run the E-step
            sigma_ss, beta_ss, bound_ss, Lambda = self.__estep_spark__(documents, beta_index, beta, 
                                                              Lambda, mu, sigma, verbose, False)
            if verbose:
                print("Completed E-Step, Iteration " +  str(i+1))

            # Run the M-step
            mu = self.__rmu_2py__(robjects.globalenv['opt.mu'](Lambda, 
                                 robjects.r("settings$gamma$mode"),
                                 robjects.r("settings$covariates$X"),
                                 robjects.r("settings$gamma$enet")))
            sigma = np.array(robjects.r("opt.sigma")(sigma_ss, 
                                                     Lambda, 
                                                     mu['mu'], 
                                                     float(robjects.r("settings$sigma$prior"))))
            beta_new_temp = robjects.globalenv['opt.beta'](self.__pybeta_ss_2r__(beta_ss), 
                                                           robjects.r('beta$kappa'), 
                                                           robjects.r('settings'))
            beta = [np.array(x) for x in beta_new_temp[0]]

            if verbose:
                print("Completed M-Step, Iteration " +  str(i+1))

        # Check for convergence
        try:
            convergence = robjects.globalenv['convergence.check'](bound_ss, convergence, 
                                                                  robjects.globalenv['settings'])
        except:
            convergence = robjects.globalenv['convergence.check'](bound_ss, 
                                                                  robjects.globalenv['convergence'], 
                                                                  robjects.globalenv['settings'])

        # Print updates if we have not converged
        stopits = bool(self.__rlist_2py__(convergence)['stopits'][0])
        if stopits is not True and verbose:
            robjects.r.report(convergence, robjects.r.ntokens, self.__pybeta_2r__(beta),
                     robjects.r.vocab, robjects.r('settings$topicreportevery'),
                     verbose);

        # Increase counter
        i += 1
        if verbose:
            print("")

    # Some book-keeping
    self.em_time = time.clock() - start_time
    self.trained = True

    # Save the results
    self.beta = beta
    self.logbeta = [np.log(x) for x in self.beta]
    self.Lambda = Lambda
    self.mu = mu
    self.sigma = sigma
    self.settings = settings
    self.vocab = robjects.r.vocab
    self.convergence = convergence
    self.theta = np.array(robjects.r("theta_from_lambda")(Lambda))
    self.eta = np.array(robjects.r("eta_from_lambda")(Lambda))
    self.invsigma = np.linalg.inv(sigma)